In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import ttk

root = tk.Tk()
root.title("Sudoku Solver")

class SudokuSolver:
    def __init__(self, grid):
        self.grid = grid

    def is_valid(self, row, col, num):
        if (num in self.grid[row, :]) or (num in self.grid[:, col]):
            return False
        block_row = row // 3
        block_col = col // 3
        block_grid = self.grid[block_row*3:(block_row+1)*3, block_col*3:(block_col+1)*3]
        if num in block_grid:
            return False
        return True

    def solve(self, row=0, col=0):
        if row == 9:
            return True
        next_row = row + 1 if col == 8 else row
        next_col = (col + 1) % 9
        if self.grid[row, col] != 0:
            return self.solve(next_row, next_col)
        numbers = list(range(1, 10))
        np.random.shuffle(numbers)
        for num in numbers:
            if self.is_valid(row, col, num):
                self.grid[row, col] = num
                if self.solve(next_row, next_col):
                    return True
                self.grid[row, col] = 0
        return False

class SudokuVisualizer:
    def __init__(self, grid):
        self.grid = grid

    def visualize(self, canvas, entries):
        canvas.delete("all")
        cell_size = 30
        for i in range(9):
            for j in range(9):
                x0 = j * cell_size
                y0 = i * cell_size
                x1 = x0 + cell_size
                y1 = y0 + cell_size
                canvas.create_rectangle(x0, y0, x1, y1, outline="black")
                num = self.grid[i, j]
                if num != 0:
                    x_center = (x0 + x1) / 2
                    y_center = (y0 + y1) / 2
                    canvas.create_text(x_center, y_center, text=str(num), font=("Arial", 16))
                else:
                    entry = tk.Entry(canvas, justify="center", font=("Arial", 16), width=2)
                    entry.grid(row=i, column=j)
                    entries[i][j] = entry

def generate_and_visualize_sudoku(entries):
    difficulty = difficulty_var.get()
    grid = generate_sudoku(difficulty)
    visualizer.grid = grid
    visualizer.visualize(canvas, entries)

def submit_sudoku(entries):
    for i in range(9):
        for j in range(9):
            entry = entries[i][j]
            try:
                num = int(entry.get())
                if 1 <= num <= 9:
                    grid[i, j] = num
                else:
                    entry.delete(0, tk.END)
            except ValueError:
                entry.delete(0, tk.END)
    solver = SudokuSolver(grid.copy())
    if solver.solve():
        visualizer.grid = solver.grid
        visualizer.visualize(canvas, entries)
    else:
        print("No solution found.")

def generate_sudoku(difficulty):
    grid = np.zeros((9, 9), dtype=int)
    solver = SudokuSolver(grid)
    solver.solve()
    if difficulty == 'easy':
        num_spaces = 20
    elif difficulty == 'medium':
        num_spaces = 30
    elif difficulty == 'hard':
        num_spaces = 40
    for i in range(num_spaces):
        row, col = np.random.randint(9, size=2)
        while grid[row, col] == 0:
            row, col = np.random.randint(9, size=2)
        grid[row, col] = 0
    return grid

def clear_canvas(canvas):
    canvas.delete("all")

grid = generate_sudoku(difficulty='medium')
visualizer = SudokuVisualizer(grid)
entries = [[None for _ in range(9)] for _ in range(9)]

frame = ttk.Frame(root)
frame.pack(padx=10, pady=10)

difficulty_label = ttk.Label(frame, text="Select Difficulty:")
difficulty_label.grid(row=0, column=0, padx=5, pady=5)

difficulty_var = tk.StringVar()
difficulty_dropdown = ttk.Combobox(frame, textvariable=difficulty_var, values=("easy", "medium", "hard"))
difficulty_dropdown.grid(row=0, column=1, padx=5, pady=5)
difficulty_dropdown.set("medium")

generate_button = ttk.Button(frame, text="Generate Sudoku", command=lambda: generate_and_visualize_sudoku(entries))
generate_button.grid(row=0, column=2, padx=5, pady=5)

submit_button = ttk.Button(frame, text="Submit Sudoku", command=lambda: submit_sudoku(entries))
submit_button.grid(row=0, column=3, padx=5, pady=5)

quit_button = ttk.Button(frame, text="Quit", command=root.quit)
quit_button.grid(row=1, column=0, columnspan=4, padx=5, pady=10)

canvas = tk.Canvas(root, width=270, height=270)
canvas.pack(padx=10, pady=10)
clear_canvas(canvas)

root.mainloop()

: 